# Práctica 10

# Perceptrón multicapa

## Santiago Blasco Arnaiz

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
import time

In [2]:
XRaw, Y = fetch_openml(name='thyroid-allhypo', return_X_y = True)

In [3]:
#Creamos el escalador y escalamos los datos
scaler = MinMaxScaler()
X = scaler.fit_transform(XRaw)

In [4]:
X.shape, Y.shape

((2800, 26), (2800,))

In [5]:
start = 10
stop = 100
step = 10

#Tamaños de la capa oculta
sizes = np.arange(start,stop + step, step)
sizes

array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100])

In [6]:
#Creamos un dataframe para introducir toda la información y mostrarla
results = pd.DataFrame(columns = ['Tamaño', 'K', 'Entrena','Verifica','T. Ejecución'])

index = 0

for size in sizes:

    #Número de carpetas
    K = 10

    #Generamos las particiones para cross validation
    skf = KFold(n_splits = K)

    #Contador para variar la carpeta
    k = 0

    #Iterador de las carpetas de Cross Validation
    for train_index, test_index in skf.split(X,Y):
        
        X_test, Y_test = X[test_index], Y[test_index]
        X_train, Y_train = X[train_index], Y[train_index]
        
        #Tomamos la medida de tiempo
        time0 = time.time()
        
        #Creamos el clasificador
        clf = MLPClassifier(hidden_layer_sizes = size, max_iter = 2000)
        
        #Entrenamos
        clf.fit(X_train, Y_train)
        
        trainScore = clf.score(X_train, Y_train)
        testScore = clf.score(X_test, Y_test)
        
        timeExec = time.time() - time0
        
        #Añadimos los datos obtenidos al dataframe de la solución
        results.loc[index,:] = [size, k, trainScore, testScore, timeExec]

        k += 1
        index += 1

In [7]:
resultsArray = np.asarray(results)

In [8]:
summary = pd.DataFrame(columns = ['Tamaño oculta', 'Entrena', 'Verifica','T. Ejecución'])

for i in range(K):
    down, up = i*10,(i+1)*10
    printArray = resultsArray[down:up,:]
    print('Tamaño de la capa oculta: ',printArray[0][0],'\n')
    print('Fold \tEntrena \tVerifica \tT. Ejecución')
    for p in printArray:
        print(p[1],' \t','%.4f' % p[2],' \t','%.4f' % p[3],' \t','%.4f' % p[4])
    print('\n')
    means = np.mean(printArray, axis = 0)
    timeExec = printArray[:,4]
    print('MEDIA (Verifica) = %.4f' % means[3])
    print('MEDIA (Entrena) = %.4f' % means[2])
    print('Tiempo total (seg) = %.4f' % np.sum(timeExec))
    print('\n')
    
    summary.loc[i,:] = [printArray[0][0], means[2], means[3], np.sum(timeExec)]

Tamaño de la capa oculta:  10 

Fold 	Entrena 	Verifica 	T. Ejecución
0  	 0.7298  	 0.6929  	 2.6264
1  	 0.7262  	 0.6964  	 2.4329
2  	 0.7266  	 0.6679  	 2.2251
3  	 0.7179  	 0.7179  	 2.2420
4  	 0.7254  	 0.6750  	 1.9780
5  	 0.7139  	 0.7429  	 1.9411
6  	 0.7179  	 0.7393  	 2.5861
7  	 0.7234  	 0.7000  	 2.2143
8  	 0.7202  	 0.7214  	 2.2248
9  	 0.7183  	 0.7750  	 1.9755


MEDIA (Verifica) = 0.7129
MEDIA (Entrena) = 0.7219
Tiempo total (seg) = 22.4462


Tamaño de la capa oculta:  20 

Fold 	Entrena 	Verifica 	T. Ejecución
0  	 0.7290  	 0.7071  	 2.1005
1  	 0.7302  	 0.7000  	 2.4278
2  	 0.7294  	 0.6714  	 2.2927
3  	 0.7254  	 0.7214  	 2.4749
4  	 0.7266  	 0.6750  	 2.0024
5  	 0.7151  	 0.7464  	 1.6837
6  	 0.7206  	 0.7500  	 2.0233
7  	 0.7313  	 0.6893  	 1.4689
8  	 0.7270  	 0.7143  	 2.1304
9  	 0.7159  	 0.7750  	 1.8211


MEDIA (Verifica) = 0.7150
MEDIA (Entrena) = 0.7250
Tiempo total (seg) = 20.4257


Tamaño de la capa oculta:  30 

Fold 	Entrena 	Verif

In [9]:
summaryArray = np.asarray(summary)

In [10]:
print('Tamaño oculta \tEntrena \tVerifica \tT. Ejecución')
for s in summaryArray:
    print(s[0],' \t\t','%.4f' % s[1],' \t','%.4f' % s[2],' \t','%.4f' % s[3])

Tamaño oculta 	Entrena 	Verifica 	T. Ejecución
10  		 0.7219  	 0.7129  	 22.4462
20  		 0.7250  	 0.7150  	 20.4257
30  		 0.7285  	 0.7114  	 22.0061
40  		 0.7274  	 0.7100  	 21.9417
50  		 0.7304  	 0.7107  	 20.9677
60  		 0.7328  	 0.7118  	 24.2596
70  		 0.7319  	 0.7104  	 22.0274
80  		 0.7321  	 0.7132  	 23.4993
90  		 0.7329  	 0.7064  	 27.9641
100  		 0.7341  	 0.7129  	 25.8944


In [11]:
bestResult = np.argmax(summaryArray[:,2])

print('El mejor resultado (%.4f)' % summaryArray[bestResult][2], 'se consigue con tamaño %i para la capa oculta.' % summaryArray[bestResult][0])

El mejor resultado (0.7150) se consigue con tamaño 20 para la capa oculta.


In [12]:
bestTime = np.argmin(summaryArray[:,3])

print('El mejor tiempo (%.4f)' % summaryArray[bestTime][3], 'se consigue con tamaño %i para la capa oculta.' % summaryArray[bestTime][0])

El mejor tiempo (20.4257) se consigue con tamaño 20 para la capa oculta.
